In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
core_df = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\data\structured_course_data.xlsx")
core_df

In [ ]:
core_df['cleaned race names'] = core_df['Race Name'].str.replace(r'\b\d{4}\b', '', regex=True)

In [ ]:
core_df['cleaned race names'] = core_df['cleaned race names'].str.replace('Stage', '', regex=False)

In [ ]:
core_df['cleaned race names'] = core_df['cleaned race names'].str.replace(r'\d+', '', regex=True)

In [ ]:
cleaned_names = core_df[['cleaned race names']]
cleaned_names

In [ ]:
cleaned_names.to_excel('cleaned_names.xlsx')

In [ ]:
cleaned_names['cleaned race names'] = cleaned_names['cleaned race names'].str.replace('-', '')

In [ ]:
cleaned_names

In [ ]:
# going to try and pull just the race names themselves from the PCS api

In [ ]:
pip install procyclingstats

In [ ]:
import procyclingstats as pcs

In [ ]:
core_df['Year'] = core_df['Race Name'].str.extract(r'(\d{4})')

In [ ]:
core_df['Year'].value_counts()

In [ ]:
core_2017 = core_df[core_df['Year'] == 2017]
core_2018 = core_df[core_df['Year'] == 2018]
core_2019 = core_df[core_df['Year'] == 2019]
core_2020 = core_df[core_df['Year'] == 2020]
core_2021 = core_df[core_df['Year'] == 2021]
core_2022 = core_df[core_df['Year'] == 2022]
core_2023 = core_df[core_df['Year'] == 2023]

In [ ]:
core_df.info()

In [ ]:
core_df['Year'] = core_df['Year'].astype(int)

In [ ]:
core_2017

In [ ]:
core_2017.to_excel('core_2017.xlsx')
core_2018.to_excel('core_2018.xlsx')
core_2019.to_excel('core_2019.xlsx')
core_2020.to_excel('core_2020.xlsx')
core_2021.to_excel('core_2021.xlsx')
core_2022.to_excel('core_2022.xlsx')
core_2023.to_excel('core_2023.xlsx')

In [ ]:
import requests
import csv
import time

def fetch_race_data(url, year, circuit):
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.text, 'html.parser')
    race_table = soup.find('table', {'class': 'basic'})
    
    if not race_table:
        return []

    races = []
    for row in race_table.find_all('tr')[1:]: 
        cols = row.find_all('td')
        if len(cols) < 4:
            continue

        date = cols[0].get_text(strip=True)
        race_name_tag = cols[1].find('a')
        race_name = race_name_tag.get_text(strip=True) if race_name_tag else cols[1].get_text(strip=True)

        race_classification = cols[-1].get_text(strip=True)

        winner = cols[2].get_text(strip=True) if len(cols) > 2 else "N/A"
        race_class = cols[3].get_text(strip=True) if len(cols) > 3 else "N/A"

        races.append([year, circuit, date, race_name, race_classification, winner, race_class])

    return races

base_url = "https://www.procyclingstats.com/races.php"
years = range(2017, 2024)
circuits = [1, 2, 11, 12, 13, 14, 15, 16, 17, 18, 21, 24]

all_races = []

for year in years:
    for circuit in circuits:
        url = f"{base_url}?year={year}&circuit={circuit}&class=&filter=Filter"
        races = fetch_race_data(url, year, circuit)
        all_races.extend(races)

with open('procyclingstats_races_2017_2023.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Year', 'Circuit', 'Date', 'Date 2', 'Classification', 'Race Name', 'Winner'])
    writer.writerows(all_races)

print("Extraction complete")

In [ ]:
def fetch_race_data(url, year, circuit):
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.text, 'html.parser')
    race_table = soup.find('table', {'class': 'basic'})
    
    if not race_table:
        return []

    races = []
    for row in race_table.find_all('tr')[1:]: 
        cols = row.find_all('td')
        if len(cols) < 4:
            continue

        date = cols[0].get_text(strip=True)
        race_name_tag = cols[1].find('a')
        race_name = race_name_tag.get_text(strip=True) if race_name_tag else cols[1].get_text(strip=True)

        race_classification = cols[-1].get_text(strip=True)

        winner = cols[2].get_text(strip=True) if len(cols) > 2 else "N/A"
        race_class = cols[3].get_text(strip=True) if len(cols) > 3 else "N/A"

        races.append([year, circuit, date, race_name, race_classification, winner, race_class])

    return races

base_url = "https://www.procyclingstats.com/races.php"
years = range(2017, 2024)
circuits = [1]

all_races = []

for year in years:
    for circuit in circuits:
        url = f"{base_url}?year={year}&circuit={circuit}&class=&filter=Filter"
        races = fetch_race_data(url, year, circuit)
        all_races.extend(races)

with open('procyclingstats_UCIworldtour_2017_2023.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Year', 'Date', 'Date 2', 'Classification', 'Race Name', 'Winner'])
    writer.writerows(all_races)

print("Extraction complete")